In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [4]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3b
  atmpro = trp
  band = 3b
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.55, 1.5, 1.66)]
  wgt = [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD             CLIRAD `coolr_bands`           
            layer      coolrg                layer     coolrg
pressure                                                     
0.000312        1  103.114210                    1  61.102395
0.000750        2   35.448611                    2  43.824362
0.001052        3   33.863281                    3  42.492908
0.001476        4   30.439658                    4  38.732224
0.002070        5   26.875648                    5  32.955375
0.002904        6   23.797446                    6  26.421538
0.004074        7   21.333807                    7  20.858446
0.005714        8   19.419150                    8  17.585303
0.008015        9   17.910940                    9  16.490188
0.011243       10   16.650048                   10  16.524072
0.015771       11   15.513231                   11  16.646411
0.022122       12   14.460524                   12  16.279381
0.031031       13   13.654870                   13  15.386054
0.043528       14   12.577798                   14  13.592952
0.061057       15   10.980415                   15  11.091420
0.085645       16    9.631641                   16   9.192465
0.120136       17    8.820962                   17   8.481754
0.168516       18    8.391238                   18   8.582934
0.236378       19    8.267057                   19   8.967606
0.331549       20    8.458176                   20   9.371026
0.465100       21    9.302119                   21   9.980026
0.652400       22    8.872362                   22   8.960153
0.915100       23    6.864891                   23   6.356061
1.283650       24    5.258003                   24   4.641653
1.800600       25    4.196110                   25   4.101027
2.525700       26    3.374341                   26   3.705548
3.542800       27    2.647236                   27   2.958665
4.969550       28    2.129312                   28   2.139121
6.970850       29    1.881599                   29   1.853180
9.778100       30    1.664621                   30   1.881763
13.715850      31    1.378541                   31   1.727430
19.239350      32    1.031640                   32   1.213742
26.987250      33    0.806354                   33   0.793367
37.855300      34    0.705175                   34   0.736993
53.100050      35    0.552936                   35   0.676831
73.887500      36    0.401556                   36   0.460287
97.662500      37    0.302719                   37   0.252452
121.437500     38    0.214930                   38   0.131755
145.212500     39    0.143897                   39   0.074725
168.987500     40    0.107064                   40   0.064109
192.762500     41    0.080846                   41   0.059261
216.537500     42    0.059400                   42   0.053033
240.312500     43    0.038823                   43   0.043210
264.087500     44    0.006270                   44   0.019041
287.862500     45   -0.078236                   45  -0.065591
311.637500     46   -0.115045                   46  -0.104621
335.412500     47   -0.042184                   47  -0.032948
359.187500     48    0.015582                   48   0.019213
382.962500     49    0.029887                   49   0.027712
406.737500     50    0.035463                   50   0.030991
430.512500     51    0.038194                   51   0.033011
454.287500     52    0.039507                   52   0.032813
478.062500     53    0.040132                   53   0.033363
501.837500     54    0.040415                   54   0.032641
525.612500     55    0.040633                   55   0.032127
549.387500     56    0.040746                   56   0.031610
573.162500     57    0.040718                   57   0.031145
596.937500     58    0.040354                   58   0.030782
620.712500     59    0.039365                   59   0.029014
644.487500     60    0.037617                   60   0.028192
668.262500     61    0.035131                   61   0.025657
692.037500     62    0.032106                 

# Fluxes by Layer

CRD                           CLIRAD             \
                        flug       fldg      fnetg       flug       fldg   
pressure    level                                                          
0.000000    1     -14.099602   0.000000 -14.099602 -13.886873   0.000001   
0.000624    2     -14.101217   0.009242 -14.091976 -13.888565   0.006213   
0.000876    3     -14.102049   0.011130 -14.090920 -13.889297   0.008251   
0.001229    4     -14.103066   0.013561 -14.089505 -13.890213   0.010943   
0.001723    5     -14.104298   0.016577 -14.087721 -13.891350   0.014349   
0.002417    6     -14.105783   0.020272 -14.085511 -13.892746   0.018455   
0.003391    7     -14.107566   0.024800 -14.082767 -13.894449   0.023205   
0.004757    8     -14.109696   0.030380 -14.079316 -13.896515   0.028646   
0.006672    9     -14.112221   0.037312 -14.074910 -13.899001   0.035122   
0.009359    10    -14.115195   0.045986 -14.069209 -13.901959   0.043330   
0.013128    11    -14.118659   0.056884 -14.061775 -13.905433   0.054182   
0.018415    12    -14.122635   0.070575 -14.052060 -13.909433   0.068608   
0.025830    13    -14.127075   0.087718 -14.039357 -13.913890   0.087366   
0.036232    14    -14.131598   0.109066 -14.022532 -13.918425   0.110862   
0.050823    15    -14.135804   0.135012 -14.000792 -13.922665   0.138599   
0.071291    16    -14.139930   0.165759 -13.974170 -13.926813   0.169641   
0.100000    17    -14.144156   0.202741 -13.941415 -13.930996   0.205089   
0.140271    18    -14.148074   0.248739 -13.899335 -13.934736   0.249295   
0.196760    19    -14.150781   0.307596 -13.843185 -13.937006   0.309003   
0.275997    20    -14.150295   0.384706 -13.765589 -13.935655   0.391832   
0.387100    21    -14.141557   0.487286 -13.654271 -13.925452   0.504974   
0.543100    22    -14.106155   0.623781 -13.482375 -13.889038   0.653003   
0.761700    23    -14.033574   0.780946 -13.252627 -13.817263   0.813272   
1.068500    24    -13.940193   0.937054 -13.003139 -13.725325   0.952355   
1.498800    25    -13.834379   1.099252 -12.735128 -13.620690   1.084339   
2.102400    26    -13.715196   1.280093 -12.435103 -13.501303   1.258209   
2.949000    27    -13.584112   1.487408 -12.096704 -13.369085   1.497643   
4.136600    28    -13.454874   1.730582 -11.724292 -13.236728   1.781553   
5.802500    29    -13.341184   2.037086 -11.304098 -13.118074   2.085072   
8.139200    30    -13.232548   2.449274 -10.783274 -13.001732   2.481741   
11.417000   31    -13.121525   2.984587 -10.136937 -12.881416   3.092149   
16.014700   32    -13.027525   3.641383  -9.386142 -12.779164   3.930805   
22.464000   33    -13.021437   4.423432  -8.598005 -12.770534   4.849528   
31.510500   34    -13.138464   5.404566  -7.733898 -12.898275   5.827547   
44.200100   35    -13.312986   6.639089  -6.673897 -13.089817   7.127032   
62.000000   36    -13.514110   8.006093  -5.508018 -13.307783   8.772260   
85.775000   37    -13.690248   9.313140  -4.377108 -13.499486  10.260412   
109.550000  38    -13.765599  10.241043  -3.524556 -13.580831  11.052817   
133.325000  39    -13.785298  10.866054  -2.919244 -13.599423  11.442511   
157.100000  40    -13.807364  11.293378  -2.513986 -13.620068  11.673627   
180.875000  41    -13.835627  11.623169  -2.212458 -13.646662  11.880792   
204.650000  42    -13.872312  11.887540  -1.984771 -13.680856  12.081902   
228.425000  43    -13.921216  12.103735  -1.817482 -13.725239  12.275657   
252.200000  44    -13.991062  12.282917  -1.708145 -13.784615  12.456739   
275.975000  45    -14.124617  12.434131  -1.690486 -13.897400  12.623157   
299.750000  46    -14.505069  12.594244  -1.910825 -14.265545  12.806556   
323.525000  47    -15.199029  12.964202  -2.234827 -14.970251  13.216586   
347.300000  48    -16.014583  13.660952  -2.353630 -15.807257  13.960791   
371.075000  49    -16.828995  14.519248  -2.309747 -16.642133  14.849782   
394.850000  50    -17.631080  15.405504  -2.225577 -17.462860  15.748562 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')